In [35]:
#import libraries
import pandas as pd
import numpy as np
import io, os, csv
import glob
import datetime as dt

# Number of Missing Data Calculation

In [36]:
input_file = "USC00250075.csv"
data = pd.read_csv(input_file)

In [37]:
#covert "date" pandas format to "datetime" format 
data['DATE'] = pd.to_datetime(data['DATE'])

#separate day, month and year
data['day'] = data['DATE'].dt.day
data['month'] = data['DATE'].dt.month
data['year'] = data['DATE'].dt.year

In [38]:
#create a new dataframe
data_new = pd.DataFrame(data, columns=['RAINFALL','day', 'month', 'year'])

#### start missing count days 

In [39]:
#count of existing number of data

k = 0
month_list = []
for i in range(1981, 2021): #we specifed the years (e.g.: start =1981, end= 2021)
    if (( i%400 == 0)or (( i%4 == 0 ) and ( i%100 != 0))):
        k = 1
    #missing_count.append({'year':i})
    month_list.append(i)
    for j in range(1, 13):
        count = data_new[(data_new['year']==i) & (data_new['month']==j)]['day'].count()
        if j in [1,3,5,7,8,10,12]: #months with 31 days
            total = 31
        elif j in [4,6,9,11]: #months with 30 days
            total = 30
        else:
            if k == 1:
                total = 29 #february months with 29 days (with leapyear)
            else:
                total = 28 #february months with 28 days (without leapyear)
                
        missing = total - count #count missing data
        month_list.append(missing)

In [40]:
#create array and reshape the months
month_list = np.asarray(month_list)
month_list.shape
month_list_all = np.reshape(month_list, (40,13))

In [41]:
#year and monthwise missing data preparation and create a "pandas" dataframe
missing_number = []
missing_count = pd.DataFrame(month_list_all, columns = ['Year','Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [42]:
#create a "CSV" file
months =  ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

missing_count['Total'] = 0
missing_count['Total'] = missing_count[months].sum(axis=1)
missing_count.to_csv('missing_data.csv')